<span type="title">Java I/O 系统、枚举</span> | <span type="version"> 1.0 </span> | <span type="update">2018-08-02</span>
<span type="intro"><p class="card-text">在第一部分介绍了基础的JAVA IO API。首先介绍了File类，包括文件的遍历、过滤、创建、修改等，这是一个用来进行文件和文件夹管理的类。其次介绍了使用OutputStream、InputStream以及其导出类进行基于字节的读写的流程，介绍了Reader和Writer以及其导出类进行基于面向字符的读写的流程。在这一部分的最后介绍了RandomAccessFlie基于字节的读写。</p><p class="card-text">在第二部分，介绍了标准I/O以及其重定向的方法，介绍了JAVA1.4添加的NIO类，包括通道和Buffer的基本使用，包括通道读取的细节和缓冲器的功能，接着介绍了使用视图简化读写的方法，包括视图中的指针顺序、视图类型和缓冲器类型占用字节的对比等，最后介绍了内存映射和文件加锁的知识。</p><p class="card-text">在第三部分，介绍了使用GZIP和ZIP进行压缩和解压缩的方法，对于ZIP，介绍了使用基于ZipInputStream和ZIPFile进行操作的方法。除了压缩，还介绍了对象序列化的接口和使用流程、XML的使用流程等。</p><p class="card-text">在本章的最后，附带提到了ENUM枚举类，简要介绍了其使用方法，包括作为方法参数、for-each遍历以及switch判断，然后提到了一些高级的技巧，比如枚举类的构造器构造、自定义类方法、继承、抽象类实现等。之后介绍了枚举类基于接口的组织，介绍了枚举类的扩充：EnumSet和EnumMap，其解决了枚举类不可修改的问题。</p></span>

Java 的 IO 分为 基于适配器的IO类 和 基于 NIO 的IO类。对于基于适配器的IO类而言，其包括 InputStream、OutputStream、Reader、Writer 四种基类及其导出类，分别用于字节流的输入、输出，字符的输入、输出。此外还有 File 类用于文件和路径管理，压缩类用于文件压缩和校验，对象序列化接口和用于处理XML类的包等等。

# File 类

File 类接受一个参数的String构造，或者两个参数的String构造。在其内部，将文件夹和文件均看作一个对象。如果传入两个参数，第一个参数为此文件/文件夹的父路径，第二个参数为此文件/文件夹。如果传入一个参数，那么直接可以为一个文件夹或者文件的绝对路径。比如 File("tmp") File(".","tmp") File("C:/","hello") 分别表示当前文件夹的 tmp 文件/子文件夹，当前文件夹的 tmp 文件/子文件夹，"C:/"下的 hello 文件或者文件夹。

文件和路径可以不存在，其不影响File对象的构建。

## List & Filter

如果给定的是一个路径，那么可以通过 listFiles 或者 list 方法遍历该路径下的文件（不包含递归）。前者返回 Files 数组，后者返回 String 数组。

此外，对于这两个方法可传入 FilenameFilter 或者 FileFilter 进行过滤。一般而言，我们直接传递一个匿名内部类即可。对于此目录下任意文件对象，都会被过滤器进行一次判断。过滤器需要重载 accept 方法，其参数1为传递进来的File，参数2位这个File的Name 字符串，如果我们希望基于文件名判断，只需要对这个字符串进行正则判断即可，如果需要其余信息，比如文件类型、创建修改日期，则使用File对象（第一个参数）。此方法返回一个布尔值作为判断是否匹配的结果。

```java
//Somewhere.java -- main
File path = new File(".");
File[] files = path.listFiles();
print("Files ClASS to String: " + Arrays.toString(files));
print(Arrays.toString(path.listFiles(new Filter("src"))));

//Filter.java
class Filter implements FilenameFilter {
	private Pattern pattern;
	Filter(String regex) { pattern = Pattern.compile(regex);}
	//实现需要一个accept方法，这个方法传递进来两个参数，其一为PathName
    //其二为文件名，需要这两者进行判断过滤并返回一个bool值。
	public boolean accept(File dir, String name) {
	    printf("file is %s, name is %s\n",dir.toString(),name);
		return pattern.matcher(name).matches();
	}
}
//Somewhere2.java -- main
String[] list = path.list(); //返回当前File类包含的list的Name，不带遍历
print("List String: "  +Arrays.toString(list));
//下面是一个使用了匿名内部类的简写方式：
print(Arrays.toString(
    path.list(new FilenameFilter() {
        private Pattern pattern = Pattern.compile(".*iml");
        public boolean accept(File dir, String name) {
            return pattern.matcher(name).matches();
        }
    })
));
```

下面展示了一个Java文件过滤小工具，基于以上类和方法：

```java
class FindIt {
    public static void main(String[] args){
        String want;
        if (args.length == 0){
            System.out.printf("Usage: Use Java Regex Grammar to find files in this Folder" +
                    "\nWrite by Corkine Ma\n");
            want = "";
        } else {
            want = args[0];
            System.out.printf("I'm looking for %s\n",want);
        }
        String[] list = new File(".").list(
                new FilenameFilter() {
                    private Pattern p = Pattern.compile(want);
                    public boolean accept(File dir, String name) {
                        return p.matcher(name).matches();
                    }
                }
        );
        if (list.length == 0) {
            System.out.printf("Can't find anything match!\n");
            System.exit(0);
        } else {
            for (String s : list){
                System.out.println(s);
            }
        }

    }
}
```

## File Walk 

不像 Pthon，系统继承了一堆用于文件管理的面向过程函数，Java 的文件树遍历需要自己实现，下面的例子使用到了 listFiles 和 isDirectory 方法，如果是目录，则进行递归。

```java
class Walk {
    public static void walk(String path){
        File[] list = new File(path).listFiles();
        if (list.length != 0){
            for (File f : list) {
                if (f.isDirectory()) {
                    try {
                        walk(f.getName());
                    } catch (Exception e) {
                        printf("Can't walk %s :: %s\n",
                                f.getName(),e.toString());
                    }
                } else {
                    print(f.toString());
                }
            }
        }
    }
//result demo:
src\ENUMDemo.java
src\Hello.c
src\Hello.java
tmp\hello11575251413756461565.ma
tmp\hello15796153504553229736.ma           
```

## Create & Delete File

下面的例子展示了使用 Files 实用的方法来获取硬盘空间、创建目录、删除文件的操作。

对于创建目录，需要传递一个File实例，然后调用 mkdir， mkdirs 方法。注意，这是 File(".",".tmp") 表示的 tmp 在调用 mkdir 前并未限定其是一个文件夹，但是调用 mkdir 后，使用 isDirectory 判断即可得到 true。

对于创建文件，指定前缀、后缀、存放的目录，调用 createTempFile即可。或者对一个 File 对象使用 createFlie，这时创建的就是文件，而不是文件夹。

deleteOnExit 会在 JVM 关闭时清除文件。

```java
File f = new File(".");
printf("Disk Usage: \n" +
        "Total: %d MB\n" +
        "Free: %d MB\n",f.getTotalSpace()/1000000,f.getFreeSpace()/1000000);
File[] temp = new File[10];
new File(".","tmp").mkdir();
for (int i = 0; i < 10; i++){
    temp[i] = File.createTempFile("hello",".ma",new File("tmp"));
    if (i % 3 == 0)
    temp[i].deleteOnExit();
}
```

## Util tools

File类包含以下方法：

- 判断磁盘容量的方法：getFreeSpace, getTotalSpace, getUsableSpace

- 浏览文件的方法： list, listFiles, listRoots
- 判断和获取File属性方法: 
    - exist, equals, isFile, isHidden, isDirectory, isAbsolute, 
    - getName, getParent, getParentFile, canExecute, canRead, canWrite, lastModified
- 管理文件的方法：
    - 创建文件和目录：createNewFile, createTempFile, mkdir, mkdirs
    - 修改文件和目录属性：renameTo, setExecutable, setLastModified, setReadable, setWriteable
    - 删除文件和目录：delete, deleteOnExit

# IOStream 类

# Reader & Writer 类

InputStream 和 OutputStream 及其导出类用来作为流的读写。JAVA包含过多的类，这些类用于不同目的，比如缓冲、记录行号、文件读写、格式化等，其使用的是装饰器模式的设计，这需要使用者对其适应。

![](outputdemo.png)

![](inputdemo.png)

对于 OutputStream 而言，重要的几个装饰器分别是：

- ObjectOutputStream 用于序列化对象的输出流
- FileOutputStream 用于直接和文件相连的输出流
- ByteArrayOutputStream 用于将一串字节输出到内存的输出流
- DataOutputStream 用于将数据（各种基本类型）输出的输出流
- BufferedOutputStream 带有缓冲的输出流
- PrintStream 用于格式化的输出流

对于 InputStream 而言，重要的几个装饰器分别是：

- DataInputStream 数据输入到 int double 等变量中
- PushbackInputStream 带有反馈的输入流
- BufferedInputStream 带有缓冲的输入流
- FileInputStream 文件直接写入的输入流
- ByteArrayInputStream 字节输入流

![](readerdemo.png)

![](writerdemo.png)

对于 Reader，其和 InputStream 类似，不过是面向字符而非字节。常见的有：

- FileReader 直接读取文件的面向字符的类
- InputStreamReader 和面向字节的类交互的桥梁，所有基于字节的读取必须通过此转换成面向字符的类，才能被其余Reader使用
- BufferedReader 带有缓冲的Reader
- StreamReader 用于字符和字符串的读取，一般Reader都带有字符和字符串的读取方法，此类直接使用较少

对于 Writer，常见的有：

- FileWriter 直接文件写入
- BufferedWriter 带有缓冲的写入
- PrintWriter 较常用，格式化输出

## 常用读取操作

**使用字符流从文件读取字符**

下文展示了两种使用 BufferedReader 和 FileReader 、StringReader 从文件输入的方式。

对于文件而言，使用缓冲的优势在于，其提供比直接读取文件更快的操作，因此不论输入还是输出，只要和文件交互，基本都会通过 BufferedXXXStream。对于大段文字的读取，最好使用 StringBuilder 将每次 readLine 或者 read 得到的字符串/字符快速拼接在一起，然后就得到了完整的字符串。

此外，也展示了使用 StringReader 用于在内存中将字符流转换成字符的操作，可以直接将 BufferedReader 的内容使用 read 读取到 StringReader，然后获取字符。当然，直接从 char 流中获取字符甚至字符串，甚至整行都可以。

```java
public static String read(String filename){
    try {
    BufferedReader bf = new BufferedReader(new FileReader(filename));
    StringBuilder sb = new StringBuilder();
    String s;
    while ((s = bf.readLine()) != null) {
        sb.append(s);
        sb.append("\n");
    } bf.close();
    return sb.toString();}
    catch (Exception e) {throw new RuntimeException(e);}
}
public static void main(String[] args) throws Exception {
    print(read("C:/rookery.sql"));
    //这里输入的是一个Char Stream，使用 StringReader 包装后使用 read 读取字符
    //需要注意，read返回的是int，需要转换成char
    StringReader sr = new StringReader(read("C:/rookery.sql"));
    int c;
    while ((c = sr.read()) != -1) {
        System.out.print((char)c);
    }
}
```

**使用字节流从文件读取字符**

下文展示了 FileInputStream、BufferedInputStream、ByteArrayInputStream、DataInputStream 交互的操作。

1、展示了将内存中的字节发送到 ByteArrayInputStream 中，然后通过 DataInputStream 读出各种格式：int double float 等的方法。

2、展示了将文件通过字节流读取的方法： FileInputStream -> BufferedInputStream -> DataInputStream，然后通过其读取各类型的值。

使用 DataInputStream 的原因是，使用 FileInputStream 只能得到 char，显然无法直接得到类型规定大小的变量值。因此，必须通过 DIS。DIS可使用 available 判断，使用 readByte->char 读取二进制转换成 char 格式。当然，也可以直接 readChar， readFloat ...

```java
DataInputStream ds = new DataInputStream(
        new ByteArrayInputStream(StreamDemo.read("C:/rookery.sql").getBytes()));
DataInputStream ds2 = new DataInputStream(
        new BufferedInputStream((new FileInputStream("C:/rookery.sql"))));
while (ds.available() != 0){
    //注意，这里不要直接read，因为不确定什么时候遇到EOF，且各平台EOF的定义也有区别
    System.out.print((char)ds.readByte());
}
```

**字节和字符流的转换**

此外，需要注意，可以和面向字节和字符的流合并起来，通过桥梁 InputStreamReader 进行。`new BufferedReader(new InputStreamReader(new BufferedInutStream(new FileInputStream("filename.filetype")))).read()` 建立从字节文件到字符输入的关系。

## 常用写出操作

**面向字符的格式化写操作**

下面展示了使用 PrintWriter、BufferedWriter、FileWriter 写格式化数据的操作。

最简单的通过字符的写是通过 FileWriter 操作，但是其效率太低，如 `writeLikeStupid` 所示。最好添加缓冲：`BufferedWriter(new FileWriter())`。

注意，一般而言，我们需要通过非常常用的 `PrintWriter` 格式化输出，然后通向 BufferedWriter，之后是 FileWriter。对于 BufferedWriter，其可选自动刷新缓存或者使用 flush 手动更新缓存。此外，所有的写都是复写，而不是跟在后面添加数据，如果需要，使用NIO类。

当然，由于输出格式化字符到文件过于常用，因此 FileWriter 提供了直接写到文件的方法，同样需要刷新才能写入。

```java
public static void writeLikeStupid(String filename,String content){
    try {
        File f = new File(filename);
        if (!f.exists()) {f.createNewFile();}
        FileWriter fw = new FileWriter(filename);
        fw.write(content);
    } catch (Exception e) { throw new RuntimeException(e);}
}
public static void main(String[] args) throws Exception{
    //这样直接使用FileWriter的效率很低，最好添加缓冲。
    writeLikeStupid("C:/Users/Corkine/Desktop/test.log",StreamDemo.read("C:/rookery.sql").toString());
    
    PrintWriter p = new PrintWriter(
            new BufferedWriter(
        new FileWriter("C:/Users/Corkine/Desktop/test3.log")));
    p.println(StreamDemo.read("C:/rookery.sql"));
    p.flush();

    PrintWriter pw = new PrintWriter("C:/Users/Corkine/Desktop/test2.log");
    pw.println("Hello World 2");
    pw.flush(); //同样需要刷新才能写入，这就是我们的本意。
}
```

**面向字节的写操作**

同样的，DataOutputStream 和 BufferedOutputStream 和 FileOutputStream 是需要的，和读字节很类似。需要注意，读取的话，也需要按照顺序读取，否则将会出错。BufferedXXX需要刷新。

```java
DataOutputStream ds = new DataOutputStream(
        new BufferedOutputStream(
                new FileOutputStream("C:/Users/Corkine/Desktop/test3.log")));
ds.writeFloat((float)3.11423);
ds.writeInt(233); //自然，这是二进制的数据存储
ds.writeBytes("\nHello");
ds.writeUTF("Hello,Then马");
ds.flush();
//需要注意，要想将数据从二进制流中取出，必须按照格式确切的取出对应的类型！！
```

**字节和字符流的转换**

FileWriter 可接受 OutputStream 作为参数，完成字节和字符的转换。

# RandomAccessFile 类

RandomAccessFile 是一个独立的，整合了基于字节流的读和写的类。和Python open("filename","ab") 类似，可选 r、w读写。不过，RAF只能用于字节的读写，不能直接用于字符。

```java
RandomAccessFile file = new RandomAccessFile("C:/rookery.sql","r"); //只读
print(file.readLine());
print(file.readDouble());
file.close();
RandomAccessFile file2 = new RandomAccessFile("C:/Users/Corkine/Desktop/test4.log","rw"); //读写
print(file2.readLine()); //空并不会报错，返回null
file2.writeUTF("Hello World");
file2.close();
```

# 标准I/O

## 标准输入输出

System.in 是一个存粹的 InputStream 面向字节的流。因此，需要对于 in 进行包装，通过 InputSreamReader 变成基于字符的流，然后通过 BufferedReader 进行字符和字符串获取。

System.out 是一个 PrintStream 的面向字节的流。因此，可以直接通过 PrintWriter 进行字节和字符的流转换和使用。

```java
BufferedReader br = new BufferedReader(new InputStreamReader(System.in));
int i = 0;
while ((br.readLine()) != null) {
    System.out.print(++i + ": " +br.readLine() + "\n");
}
PrintWriter pw = new PrintWriter(System.out,true);
pw.println("Hello world");
//pw.flush(); 如果在构造PrintWriter设置了自动Flush,那么就不用再去Flush
```

## 重定向I/O

下文展示了重定向IO的操作：因为这里我们只是截获这些输入输出流，因此构造这些流类型到文件流的管道即可。

对于截获 System.out 流，我们使用一个包装了 FileOutputStream、BufferedOutputStream、PrintStream 的 PS 作为新的输出，使用 `System.setOut` 来截获输出流，定向到一个文件中的方法。

对于输入重定向，则需要 FileInputStream -> BufferedInputStream，通过 `System.setIn` 截获即可。

```java
PrintStream ps = System.out;
//注意，这里截获了out，以便于在之后恢复
//注意，这里将文件使用FOS通过BS和PS联系起来了，这样就可以通过setOut传递合适的类。
PrintStream out = new PrintStream(
        new BufferedOutputStream(
    new FileOutputStream("C:/Users/Corkine/Desktop/out_directing.log")),true);
System.setOut(out);
System.out.print("Hello from Console!!!马");
System.setOut(ps); //改回来输出的重定向
//注意，因为in是InputStream，必须通过桥梁InputStreamReader/FileReader再Buffered之后读取
BufferedInputStream in = new BufferedInputStream(new FileInputStream("C:/rookery.sql"));
System.setIn(in);
BufferedReader br = new BufferedReader(new InputStreamReader(System.in));
String s;
while ((s = br.readLine()) != null) {
    print(s);
}
```

PS. 对于 Python：

```python
import sys
tmp_in = sys.stdin
tmp_out = sys.stdout
sys.stdin = open("fake_in","r")
sys.stdout = open("fake_out","w")
```

# NIO

## NIO概要

Java 1.4 提供了NIO类 和 Files类，用于更快速的读写文件。这种速度的提高来自于和底层/操作系统类似的文件读写方式：使用通道和缓冲器。NIO类的结构如下所示：

![](nio.png)

文件现在通过 `FileInputStream, FileOutputStream, RandomAccessFile` 连接。这些类均有一个叫做 `getChannel` 的方法，用来返回一个 Channel。这个通道唯一交互的类是 ByteBuffer，我们只能通过 ByteBuffer 交互，然后操纵它去通道读写数据 `read, write`。这个 ByteBuffer中保存了一个带有指针的 `byte[]` 数组。我们可以通过 ByteBuffer 的 `asxxxBuffer()` 方法来获取视图，这些视图对应不同类型的数组以及指针，这样，我们就可以将多种不同数据类型通过 ByteBuffer 和 FileChannel 与 File 进行交互了。

注意，不能对一个文件即用FileInputStream，又用FileOutputStream，在这种情况下，应该使用RandomAccessFle

## NIO 简单示例

**NIO写入简单示例**

```java
FileChannel fc = new FileOutputStream("C:/Users/Corkine/Desktop/read.bin").getChannel();
//Channel通过 write 方法接受 ByteBuffer 的数据输入
// wrap 静态方法用于 将 Bytes 直接保存到一个 ByteBuffer 对象中
fc.write(ByteBuffer.wrap("Some text".getBytes()));
fc.close();//通道依然需要关闭

fc = new RandomAccessFile("C:/Users/Corkine/Desktop/read.bin","rw").getChannel();
fc.position(fc.size()); //移动到尾部
fc.write(ByteBuffer.wrap("Something new....".getBytes()));
fc.close();
```

上述示例展示了获取和一个特定文件绑定的 FileChannel，通过write方法进行写入的操作。通道的 position 方法可以指定其指针位置，以供 ByteBffer 插入或者从那个位置读取数据。此外，通道需要像文件一样关闭。

ByteBuffer 并非通过 new 方法创建，起码一般来说，通过 `wrap` 方法可以快速的将一个 `byte[]` 存放到一个新的 ByteBuffer 对象中，然后直接写入 Channel。上例通过将一个字符串转化为Bytes数组，进行了通道的写入。

**NIO读取简单示例**

对于通道而言，其需要通过 `read` 方法，将文件传送到 ByteBuffer。

对于读取而言，需要创建要给 ByteBuffer 对象。不过，这里通过 allocate 方法分配空间快速创建的，1024 byte = 1k，也就指定了这个 ByteBuffer 一次最多从通道中读取的数据长度。还可以使用 allocateDirect，这个和操作系统耦合更高，分配开支大，但是速度更快。

此外，ByteBuffer 接受数据后，其内部指针指向数据的末尾，通过 flip，可以将 limit 限制在当前的指针位置，然后将指针移动到开头，以供读取数据。使用 hasRemaining 方法 ByteBuffer 中数据是否读取完毕，使用 get 方法获取 int 格式的 Byte 数据，通过 char 转换为字符。

注意，这里只进行了一次的通道读取。对于ByteBuffer而言，进行了多次数据取出（每次1Byte）。

```java
fc = new FileInputStream("C:/Users/Corkine/Desktop/read.bin").getChannel();
ByteBuffer bb = ByteBuffer.allocate(1024); //1k
fc.read(bb);
bb.flip();
while (bb.hasRemaining()){
    System.out.print((char)bb.get());
}
```

**NIO通道传输简单示例**

```java
FileChannel in = new FileInputStream("C:/Users/Corkine/Desktop/read.bin").getChannel(),
        out = new FileOutputStream("C:/Users/Corkine/Desktop/to.bin").getChannel();
ByteBuffer b = ByteBuffer.allocate(1024);
while (in.read(b) != -1) {
    //并且，用于写入时必须flip和clear，之后才能继续read下一个单位。这里的-1继承自UNIX
    b.flip();
    out.write(b);
    b.clear();
}
//Java自带了更方便的transFrom/To, 需要指定传输位置、数量，目的管道
//可以直接连接两个通道，传输数据
in.transferTo(0,in.size(),out);
//out.transferFrom(in,0,in.size());
//现在，to保存有2倍in的内容
```

上面的例子展示了使用 ByteBuffer 在两个通道内进行文件传送的操作。注意，这里嵌套了一个 `in.read() != -1` 的循环的目的是为了将输入通道内容完全读取。对于每次读取，不用再每次取出一个Byte，而是将指针归零后直接 write 到输出通道中。需要注意，当我们输出完后进行了 clear 的操作，这个操作将 limit 限制为 capacity，将指针归零，和 flip 有一定的区别，多用于清空 ByteBuffer。

注意，Java提供了 transferTo/From 的方法，用于直接将两个通道联系起来，通过设置传入开始位置、传输长度，可以快速进行传输。

## 通道读取细节：循环判断

```java
FileChannel file = new RandomAccessFile("C:/Users/Corkine/Desktop/read.bin","rw").getChannel();
ByteBuffer bb = ByteBuffer.allocate(1024);

//传统读取数据的方法
int count;
do {
    count = file.read(bb);
    if (count != -1) {
        bb.rewind(); //pointer = 0;
        for (int i = 0; i < count; i++){
            System.out.print((char)bb.get());
        }
    }
} while (count != -1);

//另外一种读取方法（只读取到一次buffer中，其余部分未处理）
file.position(0);  // 文件归0，方便继续读取
bb.clear(); //limit = capacity， pointer = 0
file.read(bb);
bb.flip(); //limit = current pointer, pointer = 0
while ( bb.hasRemaining() ) { System.out.print((char) bb.get());}
```

注意，这里的 rewind 和 clear、flip都不同，其只是单纯的将指针归零。当读取ByteBuffer时，一共需要两层循环，其一为通道循环，多次ByteBuffer读取通道内容，其二为字符获取循环，对于每次的ByteBuffer，获取其全部字节。对于第一个循环，使用 -1 判断，对于第二个循环，使用 hasRemaining 判断 指针 和 limit 之间是否还有数据。

## 缓冲器细节：mark 和 reset

下面展示了使用 mark 和 reset 的方法，其不太有用。

```java
bb = ByteBuffer.allocate(1024);
CharBuffer cbuffer = bb.asCharBuffer();
cbuffer.put("eHllWorodl"); cbuffer.flip();
while (cbuffer.hasRemaining()){
    cbuffer.mark(); //标记
    char c1 = cbuffer.get();
    char c2 = cbuffer.get();
    cbuffer.reset(); //恢复指针为0
    cbuffer.put(c2).put(c1);//反向放置回去
} //现在指针位于3，重复3和4的交换
while (bb.hasRemaining()) {
    System.out.print(bb.getChar());
}
file.close(); //HelloWorld
```


## 视图基础

使用ByteBuffer，每次只能读出一个字节，然后进行类型强制转换。因此，考虑使用CharBuffer，但是这样的话，就需要对于Byte和Char之间进行编码转换。使用 `asCharBuffer` 获得一个 CharBuffer 对象。对于这个 Buffer，直接 get 和 put 即可。此外，我们也可以直接调用 ByteBuffer 的 get/putChar,Double,Float,Int 不通过视图直接读写特定类型的数据。

```java
bb = ByteBuffer.allocate(128);
char c;
bb.asCharBuffer().put("Hello, Again!!!");
//使用这种方式as...的put不会改变bb的指针位置，只会改变cb的位置
while ((c = bb.getChar()) != 0) {
    System.out.print(c);
    //这里使用的是bb.getChar，因此改变了bb的位置
}
bb.clear();
bb.asDoubleBuffer().put(21312.23131);
//print(bb.getDouble()); 均可
print(bb.asDoubleBuffer().get());
```

使用视图需要注意ByteBuffer和视图调用情况不同时的指针位置问题：

- 1、通过as...调用的，ByteBuffer 位置总是不变，除非调用 ByteBuffer 自身非指定index的方法（get\put）
- 2、通过子buffer 调用的，子buffer 位置当调用 非index put/get，则位置变化，当调用 index put/get，则位置不变
- 3、在上述2中，ByteBuffer 的位置，除非显式调用 ByteBuffer.getChar etc 则其指针位置总和子buffer 独立。


```java
bb.clear();
IntBuffer ib = bb.asIntBuffer();
ib.put(new int[]{11,112,231,4321,55,4,6,567,657});
printf("\nib POSITION: %d\n",ib.position()); //ib因为put，位置变化
printf("\nBB POSITION: %d\n",bb.position()); //但是，因为没有直接操纵ib，因此bb位置不变
print(ib.get(3)); // ib因为指定get的index，所以其指针位置不变
printf("\nPOSITION: %d\n",ib.position());
ib.put(4,233); //同理，如果指定将某个元素放在某个位置，则指针位置不需改变
printf("\nPOSITION: %d\n",ib.position());
ib.rewind();
printf("\nPOSITION: %d\n",ib.position()); //通过rewind后，ib位置变化
ib.get();
printf("\nPOSITION: %d\n",ib.position()); //通过get后，ib位置变化
printf("\nPOSITION: %d\n",bb.position()); //而bb和ib独立，位置不变
print(bb.getInt());
printf("\nPOSITION: %d\n",bb.position()); //除非显式调用了bb的方法
// 并且看到，bb的步进和ib的步进不同
print(bb.getInt());
printf("\nPOSITION: %d\n",bb.position()); //bb position 现在为8
print(ib.get ());
printf("\nPOSITION: %d\n",ib.position()); //ib步进为1
```

此外，需要注意，对于视图的指针步进和对于 ByteBuffer 的指针步进不同。


## 视图细节

需要注意，对于一个 ByteBuffer，当从一个 ByteBuffer 新建一个视图的时候，这个视图将会从 ByteBuffer 指针所指的位置开始(也就是说这个视图最开始的位置指向 ByteBuffer 当前的位置)，但是其丢弃了其余任何关于 ByteBuffer 的东西，比如 mark 和 limit，对于这个视图的修改会反映在 ByteBuffer 上。因此，asCharBuffer 调用一般需要重置指针后再使用视图：`bb.rewind().asCharBuffer();`

```java
//视图缓冲器中，要给byte数组，可能表示不同的含义，比如：
bb = ByteBuffer.wrap(new byte[]{0,0,0,0,0,0,0,'a'});
while (bb.hasRemaining()) {
    System.out.print(bb.position() + "-->" + bb.get() + "; \n");
}
CharBuffer cb = bb.rewind().asCharBuffer();
while (cb.hasRemaining()){
    System.out.print(cb.position() + "-->" + cb.get() + "; \n");
}
IntBuffer ib2 = bb.rewind().asIntBuffer();
while (ib2.hasRemaining()){
    System.out.print(ib2.position() + "-->" + ib2.get() + "; \n");
}
```
![](niodemo.png)

可以看到，不同的字节和字符位置并非一一对应的。因此，对于普通使用，最好选用特定的我们需要的视图，对视图的指针进行遍历。而不要直接使用Buffer，这样可能导致问题。

## 内存映射文件：MappedByteBuffer 

我们可以直接将文件映射到 MappedByteBuffer 上，这个 Buffer 是 ByteBuffer 的一个导出类。使用方法类似，不过需要传入参数用于读写、指定Map的长度。

```java
//我们在这里设置128MB最大，MBB是BB的一个继承类，各种方法均可使用。注意，这种方式创建开销大，但是
//速度很快
MappedByteBuffer f = new RandomAccessFile("C:/Users/Corkine/Desktop/new.large","rw")
        .getChannel().map(FileChannel.MapMode.READ_WRITE,0,0x8FFFFFF);
for (int i = 0; i < 0x8ffffff; i++) {
    f.put((byte)'x');
}
print("The position now is: " + f.position());
for (int i = 0; i < 0x8ffffff/10 + 4; i++){
    System.out.print((char)f.get(i));
}
```

## 文件加锁

通道可以进行加锁，这会映射到操作系统上，以避免其余程序对文件的操作。

使用 FileLock 类对于 FileChannel 进行加锁。使用Lock阻塞进程，直到获取锁，知道其线程中断或者通道关闭。而tryLock如果不能获得，则直接返回。其参数可指定加锁的范围：位置、长度、是否共享锁。

```java
//展示了通过通道进行文件阻塞和非阻塞加锁的机制
FileOutputStream file = new FileOutputStream("C:/Users/Corkine/Desktop/read.bin");
FileChannel fc = file.getChannel();
FileLock fl = fc.tryLock();

if (fl != null) {
    print("Locking file...");
    Thread.sleep(10000);
    fl.release(); //解锁
    print("Lock released...");
}
file.close();
```

下面是一个用于部分通道加锁的示例, 从现有的MapBB中截取一段，设置需要锁定的位置，然后对这个BB以外的部分进行加锁。这里主要使用到了 position、limit、capacity 方法来获取 ByteBuffer 的指针、限制和容量。

```java
RandomAccessFile file;
{
    try {
        RandomAccessFile file = new RandomAccessFile("C:/rookery.sql", "rw");
    } catch (Exception e) {
    }
}
FileChannel fc = file.getChannel();
void demoLock(ByteBuffer map, int start, int end) throws Exception {
    map.limit(end);
    map.position(start);
    lockExcept(map.slice());
}
void lockExcept(ByteBuffer remain) throws Exception {
    FileLock f1 = fc.tryLock(0,remain.position(),false);
    FileLock f2 = fc.tryLock(remain.limit(),remain.capacity(),false);
    //do something
    f2.release();
    f1.release();
}
```

# 压缩

## GZIP 压缩

GZIP 的压缩需要在 FileOutputStream 和 BufferedOutputStream 之间创建一个 GZIPOutputStream 对象。然后使用正常的方法对 BufferedOutputStream 进行操作即可。最后得到的 FileOutputStream 就是压缩过的文件。

```java
//本代码展示了GZip数据流的压缩
BufferedReader in = new BufferedReader(
        new InputStreamReader(new FileInputStream("C:/rookery.sql")));
BufferedOutputStream out = new BufferedOutputStream(
        new GZIPOutputStream(
                new FileOutputStream("C:/Users/Corkine/Desktop/test.gz")));
int c;
while ((c = in.read()) != -1){
    out.write(c);
}
out.flush();in.close();out.close();
//这会创建一个test文件，并且将其保存在test.gz压缩包中
```

当解压的话，通过相同的反向步骤即可。

## ZIP 压缩

**压缩成为ZIP文档**

ZIP 归档文件需要使用以下步骤写入：`BufferedOutputStream -> ZipOutputStream -> CheckedSumOutputStream -> FileOutputStream` 需要注意，ZIP需要校验压缩，因此在File和ZIPOutStream之间还需要一个CheckedSumOutputStream计算MD5值。

其中 `ZipOutputStream` 调用 `.putNewEntry` 方法传入一个 ZipEntry 对象，然后将指针移动到此对象上，通过 BufferedOutputStream 输入流到 ZipOutputStream 中即可完成对于当前指针指向的这个 ZipEntry 的数据写入。这个对象本身可以设置各种文件属性，但是，不能从中读取数据，必须通过ZipOutputStream读取，这是因为从理论上来看，在解压之前，不存在这些 ZipEntry 流，所有数据都是保存在 .zip 文件中的，只是这个文件区隔了这些不同的 Entry 的部分，看起来像是这个 zip 文件是一个文件夹，其内部含有多个文件似的。


```java
FileOutputStream fos = new FileOutputStream("C:/Users/Corkine/Desktop/test.zip");
CheckedOutputStream cos = new CheckedOutputStream(fos,new Adler32());//需要声明文件流的同时声明一个校验方法
ZipOutputStream zout = new ZipOutputStream(cos);
BufferedOutputStream bos = new BufferedOutputStream(zout);

zout.setComment("Zip achieve by Corkine Ma");
zout.putNextEntry(new ZipEntry("file1.null"));
zout.putNextEntry(new ZipEntry("C:/rookery.sql")); //指针指向此处，写入到此文件中
InputStreamReader ir = new InputStreamReader(new FileInputStream("C:/rookery.sql"));
int c;
while ((c = ir.read()) != -1) {
    bos.write(c);
} bos.flush();
//zout.write(); ZOUT一般不用于写入文件，但也可以
zout.putNextEntry(new ZipEntry("file4.null")); //空文件，没有写入
bos.close(); fos.close(); zout.close(); cos.close();
```

**读取ZIP压缩文档**

同理，反向进行即可。需要对 CheckedInputStream 填入一种校验算法。读取时，对于 ZipInputStream 进行 getNextEntry 遍历，对于 ZipInputStream 进行读取（或者是与其关联的BufferedInputStream）。

```java
FileInputStream fis = new FileInputStream("C:/Users/Corkine/Desktop/test.zip");
CheckedInputStream cis = new CheckedInputStream(fis,new Adler32());
ZipInputStream zis = new ZipInputStream(cis);
BufferedInputStream bis = new BufferedInputStream(zis);
ZipEntry ze;
while ((ze = zis.getNextEntry()) != null) {
    print("Reading from " + ze);
    int i;
    while ((i = bis.read()) != -1){
        System.out.write(i);
    }
}
print("CheckSUM:" + cis.getChecksum().getValue());
```

**ZIPFile 快速读取**

此外，Java提供了 ZIPFile 类用来快速读写ZIP的方法。ZIPFile 直接传入地址即可构造。使用 `.entries` 方法得到一个枚举对象，然后使用 `hasMoreElementes` 判断枚举，使用 `nextElement()` 得到枚举，转型为 ZipEntry。

同样的，因为 ZipEntry 是抽象的文件，所以还需要对 ZipFile 进行操作，但不像 ZipInputStream，不需要直接从中读取数据，而是使用 ZipFile.getInputStream(ZipEntry) 获得这个“文件”的对应InputStream，之后再进行读写。

```java
ZipFile zf = new ZipFile("C:/Users/Corkine/Desktop/test.zip");
Enumeration e = zf.entries();
while (e.hasMoreElements()) {
    ZipEntry ze2 = (ZipEntry) e.nextElement();
    print("Find " +ze2);
    //因为zf不是流，因此需要通过getInputStream(Entry obj) 来获取对应文件的流，进行读取
    InputStreamReader isr = new InputStreamReader(zf.getInputStream(ze2));
    int i = 0;
    while ((i = isr.read()) != -1){
        System.out.write(i);
    }
}
```

# 对象序列化

可以将对象按照其在内存中的字节顺序保留下来，只用将需要保存的类继承 `Serializable` 接口即可。不用实现任何方法。如果在类中有数据或者字段不想要保存，使用 `transient` 关键字，放在返回值前，这个字段的数据在恢复后将会为 null。

使用序列化的方法很简单，使用 `ObjectOutputStream` 指定一个 `FileOutputStream` 找到一个文件用于保存即可。OOS具有以下方法：

- writeObject 写入一个对象，注意，类型将消失，被转型为Object
- writeDouble ... 等类似的方法用来保存Java内置的数据
- close 记得关闭

对于取出数据，使用 `ObjectInputStream` 的 `readObject`， `readDouble...` 方法即可。注意，对于对象，需要进行转型。下面的例子可以看到，恢复的对象的内存地址全变了。

```java
class Data implements Serializable {
    private int n;
    public Data(int n) { this.n = n;}
    public String toString() {
        return super.toString() + "KEEP: " + Integer.toString(n);
    }
}
class Worm implements Serializable {
    private Random rand = new Random(42);
    private transient Data[] d = {
            new Data(rand.nextInt(10)),
            new Data(rand.nextInt(10)),
            new Data(rand.nextInt(10))
    };
    public String toString() {
        return super.toString() + "KEEP: " + Arrays.toString(d);
    }
}
//DEBUG - MAIN
Worm w = new Worm();
ObjectOutputStream oos = new ObjectOutputStream(
        new FileOutputStream("worm.out")
);
oos.writeObject("Worn storage\n");
oos.writeObject(w);
oos.close();

ObjectInputStream ois = new ObjectInputStream(
        new FileInputStream("worm.out")
);
print((String) ois.readObject());
Worm w2 = (Worm) ois.readObject();
//可以看到，内存地址全部变了
printf("%b %b\n w: %s, \nw2: %s",w2 == w,w2.equals(w),w,w2);
```

此外，可以拦截 Sreializable 的 readObject 等方法来添加在序列化/反序列化的时候想要额外进行的操作。JAVA还有一个叫做 Externalizable 的接口用来手动指定在序列化时进行的操作。

# XML

对于 XML，最容易使用的类是 `nu.xom`，从 xom.nu 下载 `jar` 文件添加到 CLASSPATH 即可。下面是简单的介绍

```java
static Element getXML(String fname, String lname){
    Random rand = new Random();
    Element name = new Element("Name");
    Attribute a = new Attribute("id",Integer.toString(rand.nextInt(1000)));
    name.addAttribute(a);
    Element fname1 = new Element("fname");
    fname1.appendChild(fname);
    Element lname1 = new Element("lname");
    lname1.appendChild(lname);
    name.appendChild(fname1);
    name.appendChild(lname1);
    return name;
}
```

Element 代表节点，其传入 String 做为节点名称，即 `<Name>` 这样。 节点可以有属性，传入 Attribute 对象，构建一个属性，参数1为属性名称，参数2为属性值，然后对 Element 节点调用 addAttribute 添加即可，结果如下：`<Name id=2131></Name>`。

Element 可以有子节点，只用 `appendChild` 将其包裹进来即可。appendChild可以接受一个子节点，比如 `<Name><fname></fname><lname></lname></Name>` 也可以接受一个字符串，比如 `<Name>String here</Name>`。

```java
Element e = getXML("Corkine","Ma");
Element e2 = getXML("Jin","Lu");
Element e3 = getXML("Marvin","Sun");
Element root = new Element("people");
root.appendChild(e);
root.appendChild(e2);
root.appendChild(e3);
Document d = new Document(root);
print(d.toXML());
```

创建 Document 对象 `Element root = new Element("root"); Document d = new Document(root)`，传入根节点，将所有节点 appendChild 到 根节点即可，使用 `toXML` 打印XML。

```xml
<?xml version="1.0"?>
<people><Name id="239"><fname>Corkine</fname><lname>Ma</lname></Name><Name id="809"><fname>Jin</fname><lname>Liu</lname></Name><Name id="436"><fname>Marvin</fname><lname>Mu</lname></Name></people>


```

# 附录：枚举

## 枚举的基本使用

枚举可以为一堆具名的值创建一个类型，这个类声明为 final，其继承自Enum类。但是和一般类不同的是，其大部分方法，比如value，valueof都是编译器添加的。

枚举的语法很简单，几个值，用逗号隔开，放置在花括号中，使用 enum 关键字和一个类名进行限定。在使用的时候，类型为这个类名，值直接写这些值即可。

枚举的常见用途为：
- 1、用for-each循环遍历
- 2、switch语句对于值的判断
- 3、作为方法参数传递，这个方法的类型就是枚举类，其可以接受这些具名的值的传递

```java
enum Method {
    HAPPY, SAD, MAD, ANGRY
}
public class ENUMDemo {
    static void methodA(Method m){
        //枚举的作用之一为作为参数被调用
    }
    public static void main(String[] args){
        for (Method m : Method.values()){
            printf("%2$s > %1$s\n",m.name(),m.ordinal());
            //常用的方法有对于类的值进行遍历，获取实例的名称、序号
            //枚举实例同样还有equals和hashCode、CompareTo方法的实现
        }

        methodA(Method.ANGRY);
        methodA(Method.HAPPY);

        //enum自带次序，此外，这里不必声明 Method.xxx 就可以判断switch的数据值
        Method a = Method.HAPPY;
        switch (a) {
            case MAD: print("mad");
            case SAD: print("sad");
            case ANGRY:
            case HAPPY:
            default: print("something else..");
        }
    }
}
```

## 枚举的改造：构造、实现、抽象类和重载

枚举类可以做更多事情，比如对于值进行构造器构造，对于类设置内部方法，以供调用值得时候可以获得这些方法。

```java
//一般而言的enum枚举使用其自身的名字来进行区分，我们对其的要求仅仅是能够有一个类型可调用，其这几个实例能够区别开即可。
//但是enum类可以有自己的方法、构造器，虽然这较少使用。可以将这些具名实例看作一种静态方法，返回一个构造器构造的对象。
enum Method2 {
    //这些具名实例会使用构造器封装自己的行为
    HAPPY("I feel happy now!"), SAD("Sad, do you hear me?"), MAD("... mad"), ANGRY("Fuck off!");
    private String description;
    //因为在外部不可调用构造器，因此声明为 private
    private Method2(String description) {
        this.description = description;
    }
    public String getDescription() {
        return description;
    }
    //甚至toString也可以像普通类那样被覆盖
    public String toString() {
        return this.name().substring(0,1) + this.name().substring(1).toLowerCase();
    }
    public static void main(String[] args){
        Method2 x = Method2.HAPPY;
        print(x.getDescription());
        print(x.toString());
    }
}
```

你甚至可以为这个类实现一个接口（但是不能继承它）：

```java
//因为Java不直接支持多继承，但是你可以实现一个方法，就像这样。注意，使用getClass.getEnumConstants
//可以获得所有我们定义的枚举实例
enum Method3 implements Iterator<Method3> {
    HAPPY, SAD, MAD, ANGRY;
    private static int count;
    public boolean hasNext(){
        return count < this.getClass().getEnumConstants().length;
    }
    public Method3 next() {
        return this.getClass().getEnumConstants()[count++];
    }
    public static void main(String[] args){
        Method3 x = Method3.HAPPY;
        while (x.hasNext()){
            print(x.next());
        }

        //Enum.random(Method.class);
    }
}
```

同样的，你可以为这个类添加抽象方法，在每个值中实现它：

```java
//JavaSpec:可以为每个类撰写自己的方法实现，以达到自己的行为,不过，需要注意，一定要添加一个抽像方法定义。
//这和实现类的方法，然后让每个实例保存不同的数据类似。不过，这里强制使用抽象类，来让这些实例去实现，逻辑更清晰。
enum OS {
    DATE_TIME {
        String getInfo() {
            return DateFormat.getDateInstance().format(new Date());
        }
    },
    CLASSPATH {
        String getInfo() {
            return System.getenv("CLASSPATH");
        }
    };
    abstract String getInfo();
    public static void main(String[] args){
        print(OS.DATE_TIME.getInfo());
        print(OS.CLASSPATH.getInfo());
    }
}
```

此外，除了使用抽象类，此外还可以覆盖基类的方法。这里就不再展示。

## 枚举的组织

我们可以使用接口组织枚举，枚举反过来实现方法，这样就可以直接调用 Food 方法，从而使用 Food.Chinese.SICHUAN 这样的二级枚举值了。

```java
//使用接口组织枚举，以将枚举分类
interface Food {
    enum Chinese implements Food {
        SICHUAN, HENAN, WUHAN, BEIJING, ZHEJIANG
    }
    enum English implements Food {
        LONDON
    }
    enum USA implements Food {
        NEWYORK, DC
    }
}
enum Food2{;
    enum Chinese {
        SHCHUAN, HENAN, WUHAN
    }
    enum USA {
        NEWYORK, DC
    }
}
//这样维持了Food的类别，同时添加了一个子类别
Food x = Food.Chinese.WUHAN;
print(((Food.Chinese) x).ordinal());

//此外，可以嵌套enum，不过这样的类型就是子类型而不是外层类型
Food2.Chinese y = Food2.Chinese.SHCHUAN;
//Food2 y = (Food2) Food2.Chinese.SHCHUAN; 并且不可以转型
```

## 枚举的进化

我们可以用EnumSet或者EnumMap类来保存枚举数据，后者提供了修改功能（传统的枚举不能修改）。注意，EnumSet.allOf 方法可以使用类字面量创建对象，但是 EnumMap必须使用 new 关键字创建。EnumMap的Key必须为枚举值。

```java
enum Size {S, M, L, XL, XXL}

//ENUM不可删除或者增加元素，因此使用EnumSet来接受一个enum的类字面量，获得其enum实例
EnumSet<Size> es = EnumSet.allOf(Size.class);
print(es);

//同样的，我们也有只能填充key为enum实例的EnumMap类，注意，这个的创建需要使用new
EnumMap<Size,String> em = new EnumMap<>(Size.class);
em.put(Size.XXL,"Fucking fat");
em.put(Size.S,"Ohh, so thin..");
print(em);
```